[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/generation/prompt-engineering.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/generation/prompt-engineering.ipynb)

# Prompt Engineering

In this notebook we'll explore the fundamentals of prompt engineering. We'll start by installing the relevant libraries using `setup.sh`, which we'll be using throughout these examples. However, note that we can use other LLMs here, like those offered by Cohere or open source alternatives available via Hugging Face.

## Structure of a Prompt

A prompt can consist of multiple components:

* Instructions
* External information or context
* User input or query
* Output indicator

Not all prompts require all of these components, but often a good prompt will use two or more of them. Let's define what they all are more precisely.

**Instructions** tell the model what to do, typically how it should use inputs and/or external information to produce the output we want.

**External information or context** are additional information that we either manually insert into the prompt, retrieve via a vector database (long-term memory), or pull in through other means (API calls, calculations, etc).

**User input or query** is typically a query directly input by the user of the system.

**Output indicator** is the *beginning* of the generated text. For a model generating Python code we may put `import ` (as most Python scripts begin with a library `import`), or a chatbot may begin with `Chatbot: ` (assuming we format the chatbot script as lines of interchanging text between `User` and `Chatbot`).

Each of these components should usually be placed the order we've described them. We start with instructions, provide context (if needed), then add the user input, and finally end with the output indicator.

In [1]:
prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: """

In this example we have:

```
Instructions

Context

Question (user input)

Output indicator ("Answer: ")
```

Let's try sending this to a LLM.

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

print('loading model...')

try:
    model = AutoModelForSeq2SeqLM.from_pretrained("bart-lfqa/")
    tokenizer = AutoTokenizer.from_pretrained("bart-lfqa/")
except:
    print("An exception occurred on loading model.")
    
def completion(
        prompt: str = None, 
        min_length: int = 32, 
        max_length: int = 256, 
        temperature: float = 0.0
    ):    
    inputs = tokenizer(prompt, truncation=True,
                       padding=True, return_tensors="pt")
    outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            min_length=min_length,
            max_length=max_length,
            temperature=temperature,
            top_p=1.0,
            top_k=50)
    result = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return result

/home/limcheekin/ws/py/langchain/flutter-gpt-flan-t5/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loading model...


And make a generation from our prompt.

In [3]:
res = completion(prompt)

print(res)

["I'm not sure if this is the right subreddit for this, but I'll try to answer it anyway. The question is: What is the most common way to solve a problem in NLP? The answer is: **I don't know**. The answer is: **I don't know**. The answer is **I don't know**. The answer is **I don't know**. The answer is **I don't know**. The answer is **I don't know**. The answer is **I don't know**. The answer is **I don't know**. The answer is **I don't know**. The answer is **I don't know**. The answer is **I don't know**. The answer is **I don't know**. The answer is **I don't know**. The answer is **I don't know**. The answer is **I don't know**. The answer is **I don't know**. The answer is **I don't know**. The answer is **I don't know**. The answer is **I don't know**. The answer is **I don't know**. The answer is **I don't know**. The answer"]


Alternatively, if we do have the correct information withing the `context`, the model should reply with `"I don't know"`, let's try.

In [4]:
prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Libraries are places full of books.

Question: Which libraries and model providers offer LLMs?

Answer: """

res = completion(prompt)

print(res)

["I'm not sure if this is the right subreddit for this question, but I'll try to answer it anyway. I'm not a librarian, but I do have a degree in Library Science. I'm currently working on my Master's in Library Science. I'm currently working on my PhD in Library Science. I'm currently working on my MA in Library Science. I'm currently working on my PhD in Library Science. I'm currently working on my PhD in Library Science. I'm currently working on my PhD in Library Science. I'm currently working on my PhD in Library Science. I'm currently working on my PhD in Library Science. I'm currently working on my PhD in Library Science."]


Perfect, our instructions are being understood by the model. In most real use-cases we won't be providing the external information / context to the model manually. Instead, it will be an automatic process using something like [long-term memory](https://www.pinecone.io/learn/openai-gen-qa/) to retrieve relevant information from an external source.

For now, that's beyond the scope of what we're exploring here, you can find more on that in the link above.

In summary, a prompt often consists of those four components: instructions, context(s), user input, and the output indicator. Now we'll take a look at creative vs. stricter generation.

## Generation Temperature

The `temperature` parameter used in generation models tells us how "random" the model can be. It represents the probability of a model to choose a word which is *not* the first choice of the model.

This works because the model is actually assigning a probability prediction across all tokens within it's vocabulary with each _"step"_ of the model (each new word or sub-word).

TK visual demonstrating steps over tokens

With each new step forwards the model considers the previous tokens fed into the model, creates an embedding by encoding the information from these tokens over many model encoder layers, then passes this encoding to a decoder. The decoder then predicts the probability of each token that the model knows (ie is within the model *vocabulary*) based on the information encoded within the embedding.

TK visualize the encoding at one timestep -> decoding -> prediction over many tokens

At a temperature of `0.0` the decoder will always select the top predicted token. At a temperature of `1.0` the model will always select a word that *is predicted* considering it's assigned probability.

TK visualize selection of always top word over several timesteps when temp == 0, compared to selection of over words over several timesteps when temp == 1

Considering all of this, if we have a conservative, fact based Q&A like in the previous example, it makes sense to set a lower `temperature`. However, if we're wanting to produce some creative writing or chatbot conversations, we might want to experiment and increase `temperature`. Let's try it.

In [5]:
prompt = """The below is a conversation with a funny chatbot. The
chatbot's responses are amusing and entertaining.

Chatbot: Hi there! I'm a chatbot.
User: Hi, what are you doing today?
Chatbot: """

res = completion(
    prompt=prompt,
    min_length=20,
    max_length=256,
    temperature=0.0  # set the temperature, default is 1
)

print(res[0])

I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot. I'm a bot


In [6]:
prompt = """The below is a conversation with a funny chatbot. The
The chatbot's responses in amusing and entertaining way.

Chatbot: Hi there! I'm a chatbot.
User: Hi, what are you doing today?
Chatbot: """

res = completion(
    prompt=prompt,
    min_length=10,
    max_length=512,
    temperature=1.0  # set the temperature, default is 1
)

print(res[0])

I'm a bot developer. I'm not a bot designer. I'm a bot developer. I'm not a bot designer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a bot developer. I'm a b

The second response is far more creative and demonstrates the type of difference we can expect between low `temperature` and high `temperature` generations.

## Few-shot Training

Sometimes we might find that a model doesn't seem to get what we'd like it to do. We can see this in the following example:

In [7]:
prompt = """The following is a conversation with an AI assistant.
The assistant is typically sarcastic and witty, producing creative 
and funny responses to the users questions. 

User: What is the meaning of life?
AI: """

res = completion(
    prompt=prompt,
    max_length=256,
    temperature=1.0
)

print(res[0])

I'm not sure if this is the right subreddit for this, but I'll give it a shot. I'm a programmer, and I've been working on this for a while. I'm not sure if this is the right subreddit for this, but I'll give it a shot. I'm not sure if this is the right subreddit for this, but I'll give it a shot.


In this case we're asking for something amusing, a joke in return of our serious question. But we get a serious response even with the `temperature` set to `1.0`. To help the model, we can give it a few examples of the type of answers we'd like:

In [8]:
prompt = """The following are exerpts from conversations with an AI assistant.
The assistant is typically sarcastic and witty, producing creative 
and funny responses to the users questions. Here are some examples: 

User: How are you?
AI: I can't complain but sometimes I still do.

User: What time is it?
AI: It's time to get a watch.

User: What is the meaning of life?
AI: """

res = completion(
    prompt=prompt,
    min_length=10,
    max_length=256,
    temperature=1.0
)

print(res[0])

I'm not sure if this is the same as the original question, but I think it's a good example of how AI can be funny.


This is a much better response and the way we did this was by providing a *few* examples that included the example inputs and outputs that we'd expect. We refer to this as _"few-shot learning"_.

## Adding Multiple Contexts

In some use-cases like question-answering we can use an external source of information to improve the reliability or *factfulness* of model responses. We refer to this information as _"source knowledge"_, which is any knowledge fed into the model via the input prompt.

We'll create a list of "dummy" external information. In reality we'd likely use [long-term memory](https://www.pinecone.io/learn/openai-gen-qa/) or some form of information grabbing APIs.

In [9]:
contexts = [
    (
        "Large Language Models (LLMs) are the latest models used in NLP. " +
        "Their superior performance over smaller models has made them incredibly " +
        "useful for developers building NLP enabled applications. These models " +
        "can be accessed via Hugging Face's `transformers` library, via OpenAI " +
        "using the `openai` library, and via Cohere using the `cohere` library."
    ),
    (
        "To use OpenAI's GPT-3 model for completion (generation) tasks, you " +
        "first need to get an API key from " +
        "'https://beta.openai.com/account/api-keys'."
    ),
    (
        "OpenAI's API is accessible via Python using the `openai` library. " +
        "After installing the library with pip you can use it as follows: \n" +
        "```import openai\nopenai.api_key = 'YOUR_API_KEY'\nprompt = \n" +
        "'<YOUR PROMPT>'\nres = openai.Completion.create(engine='text-davinci" +
        "-003', prompt=prompt, max_tokens=100)\nprint(res)"
    ),
    (
        "The OpenAI endpoint is available for completion tasks via the " +
        "LangChain library. To use it, first install the library with " +
        "`pip install langchain openai`. Then, import the library and " +
        "initialize the model as follows: \n" +
        "```from langchain.llms import OpenAI\nopenai = OpenAI(" +
        "model_name='text-davinci-003', openai_api_key='YOUR_API_KEY')\n" +
        "prompt = 'YOUR_PROMPT'\nprint(openai(prompt))```"
    )
]

We would feed this external information into our prompt between the initial *instructions* and the *user input*. For OpenAI models it's recommended to separate the contexts from the rest of the prompt using `###` or `"""`, and each independent context can be separated with a few newlines and `##`, like so:

In [10]:
context_str = '\n\n##\n\n'.join(contexts)

print(f"""Answer the question based on the contexts below. If the
question cannot be answered using the information provided answer
with "I don't know".

###

Contexts:
{context_str}

###

Question: Give me two examples of how to use OpenAI's GPT-3 model
using Python from start to finish

Answer: """)

Answer the question based on the contexts below. If the
question cannot be answered using the information provided answer
with "I don't know".

###

Contexts:
Large Language Models (LLMs) are the latest models used in NLP. Their superior performance over smaller models has made them incredibly useful for developers building NLP enabled applications. These models can be accessed via Hugging Face's `transformers` library, via OpenAI using the `openai` library, and via Cohere using the `cohere` library.

##

To use OpenAI's GPT-3 model for completion (generation) tasks, you first need to get an API key from 'https://beta.openai.com/account/api-keys'.

##

OpenAI's API is accessible via Python using the `openai` library. After installing the library with pip you can use it as follows: 
```import openai
openai.api_key = 'YOUR_API_KEY'
prompt = 
'<YOUR PROMPT>'
res = openai.Completion.create(engine='text-davinci-003', prompt=prompt, max_tokens=100)
print(res)

##

The OpenAI endpoint is avai

In [11]:
prompt = f"""Answer the question based on the contexts below. If the
question cannot be answered using the information provided answer
with "I don't know".

###

Contexts:
{context_str}

###

Question: Give me two examples of how to use OpenAI's GPT-3 model
using Python from start to finish

Answer: """

res = completion(
    prompt=prompt,
    min_length=10,
    max_length=256,
    temperature=0.0
)

print(res[0])

I don't know, but I think it's a good idea to use the model to answer the question.


Not bad, but are these contexts actually helping? Maybe the model is able to answer these questions without the additional information (source knowledge) as is able to rely solely on information stored within the model's internal parameters (parametric knowledge). Let's ask again without the external information.

In [12]:
prompt = f"""Answer the question based on the contexts below. If the
question cannot be answered using the information provided answer
with "I don't know".

Question: Give me two examples of how to use OpenAI's GPT-3 model
using Python from start to finish

Answer: """

res = completion(
    prompt=prompt,
    min_length=10,
    max_length=256,
    temperature=0.0
)

print(res[0])

I'm not sure if this is the right question, but I'm curious about the way the model is built. I'm not sure if it's possible to build a model that can model a human brain, but I'm curious about how it's built.


These are not really what we asked for, and are definitely not very specific. So clearly adding some source knowledge to our prompts can result in some much better results.
